In [1]:
import numpy as np
from random import random, seed
from copy import deepcopy
from queue import PriorityQueue, deque

## Simple functions from the first classes
This is left just as an example (to compare the behaviour).

In [ ]:
def print_matrix(vertices, matrix):
  """
  Printing a graph given by adjacency matrix
  """
  n = len(matrix)
  if (vertices is not None) and (len(vertices) == n):
    vv = vertices
  else:
    vv = range(1, n+1)
  for i in range(n):
    print(vv[i], ":", end="")
    for j in range(n):
      if matrix[i, j]:
        print(" ", vv[j], end="")
    print("")

def print_dict(graph):
  """
  Printing of a graph (given as a dictionary/neighbouring list)
  """
  for v in graph:
    print(v, ":", end="")
    for u in graph[v]:
      print(" ", u, end="")
    print("")

## Class *Graph*

In [6]:
class Graph:
    def __init__(self, graph=None):
        if graph is None:
            graph = {}
        self.graph = graph

    # dict initializer
    @classmethod
    def from_dict(cls, graph):
        return cls(graph)

    # array initializer
    @classmethod
    def from_matrix(cls, matrix, vertices = None):
        if (vertices is None) or (len(vertices) != len(matrix)):
            vertices = [*range(1, len(matrix) + 1)]
        return cls.from_dict(cls._matrix_to_dict(matrix, vertices))

    # two private methods matrix <-> dictionaries
    def _matrix_to_dict(matrix, vertices: list) -> dict:
        """
        Converts a graph given as an adjacency matrix to a graph in dict form.
        """
        res_dict = {}
        for i, v in enumerate(vertices):
            neighbours = [vertices[j] for j, edge in enumerate(matrix[i]) if edge]
            res_dict[v] = neighbours
        return res_dict

    def _dict_to_matrix(self, _dict: dict) -> np.array:
        """
        Converts a graph in dict form to its adjacency matrix.
        """
        n = len(_dict)
        vertices = [*_dict.keys()]
        matrix = np.zeros(shape = (n, n), dtype=int)
        for u,v in [
            (vertices.index(u), vertices.index(v))
            for u, row in _dict.items() for v in row
        ]:
            matrix[u][v] += 1
        return matrix

    def vertices(self) -> list:
        """
        Returns list of vertices of the graph.
        """
        return [*self.graph.keys()]

    def matrix(self) -> np.array:
        """
        Returns the adjacency matrix of the graph.
        """
        return self._dict_to_matrix(self.graph)

    # redefinition of print for objects of class Graph
    def __str__(self):
        res = ""
        for v in self.graph:
            res += f"{v}:"
            for u in self.graph[v]:
                res += f" {u}"
            res += "\n"
        return res

    # The following is for free thanks to the above
    def to_neighbourlist(self, filename: str):
        """
        Saves a graphs to a text file as a neighbour dict.\n
        Filename is a file path.
        """
        file = open(filename, "w")  # open textfile for writing
        file.write(str(self))
        file.close()

    # Modyfying graphs
    def add_vertex(self, vertex):
        """
        Adds a new vertex to the graph.
        """
        if vertex not in self.graph:
            self.graph[vertex] = []

    def del_vertex(self, vertex):
        """
        Removes a vertex from the graph.
        """
        if vertex in self.graph:
            self.graph.pop(vertex)
            for u in self.graph:
                if vertex in self.graph[u]:
                    self.graph[u].remove(vertex)

    def add_arc(self, arc):
        """
        Given pair of vertices (arc variable) add an arc to the graph
        We consider simple, directed graphs.
        """
        u, v = arc
        self.add_vertex(u)
        self.add_vertex(v)
        if v not in self.graph[u]:
            self.graph[u].append(v)

    def add_edge(self, edge: list):
        """
        Given pair of vertices (edge variable) add an edge to existing graph.
        We consider simple, undirected graphs, as symmetric digraphs without loops.
        """
        u, v = edge
        if u == v:
            raise ValueError("Loops are not allowed!")
        self.add_vertex(u)
        self.add_vertex(v)
        if v not in self.graph[u]:
            self.graph[u].append(v)
        if u not in self.graph[v]:
            self.graph[v].append(u)

    # reading from a file
    @staticmethod
    def from_edges(filename: str, directed = 0):
        """
        Read the graph from file, that in each line contains either
        the description of a vertex (one word) or
        the description of an edge/arc (at least 2 words).
        The resulting graph is returned as a neighbourhood list.
        Variable "filename" contains the whole path to the file.
        """
        graph = Graph()
        file = open(filename, "r")          # open the file to read
        for line in file:                   # for each line of the file
          words = line.strip().split()      # splits the line into words
          if len(words) == 1:               # one word - vertex description
            graph.add_vertex(words[0])
          elif len(words) >= 2:             # at least two words, first two are the edge description
            if directed:
              graph.add_arc([words[0], words[1]])
            else:
              graph.add_edge([words[0], words[1]])
        file.close()
        return graph

    @staticmethod
    def random_graph(n: int, p: float):
        """
        Creates a random graph in G(n, p) model.
        """
        rand_graph = Graph()
        for i in range(1, n + 1):
            rand_graph.add_vertex(i)
            for j in range(1, i):
                if random() < p:
                    rand_graph.add_edge([i, j])
        return rand_graph

    @staticmethod
    def cycle(n: int):
        """
        Creates a cycle C_n on n vertices
        """
        cycle = Graph()
        for i in range(n-1):
          cycle.add_edge([i+1, i+2])
        cycle.add_edge([1, n])
        return cycle


    def Prufer(self):
      """
      Returns the Prufer code of a tree.
      It is necessary that the graph is a tree (it is not checked).
      Result is given as a string (empty for trees on 1 or 2 vertices).
      """
      tr = deepcopy(self.graph)   # copy of a tree, as we destroy it
      code = ""
      for i in range(len(self.graph) - 2):
        for x in sorted(tr):
          if len(tr[x]) == 1:   # least leaf
            break
        v = tr[x][0]            # the unique neighbour of x
        code = code + f"{v} "
        tr[v].remove(x)         # remove x from neighbours of v
        tr.pop(x)               # remove x from the tree
      return code.strip()

    @staticmethod
    def tree_from_Prufer(code: str):
        """
        Creating a tree from a Prufer code.
        """
        tree = Graph()
        clist = [int(x) for x in code.strip().split()]   # code as a list of numbers
        n = len(clist) + 2                  # number of vertices
        vert = [*range(1, n+1)]             # list of numbers 1..n
        for v in vert:
          tree.add_vertex(v)
        for i in range(n-2):
          for x in vert:
            if not x in clist:    # x - least leaf
              break
          v = clist.pop(0)    # remove the first element from the code - the neighbour of x
          tree.add_edge((x, v))
          vert.remove(x)
        tree.add_edge(vert)
        return tree


    def connected_components(self):
      """
      Looks for connected components of undirected graph.
      Returns a list of its vertex-sets.
      Remark: the first element contains the set of all graph vertices
      """
      def DFS(u):
        """
        Deep first search (as internal method).
        """
        for w in self.graph[u]:
          if w not in VT[0]:      # w - not visited yet
            VT[0].add(w)          # already visited
            VT[-1].add(w)         # w - in the last connected component
            DFS(w)

      """
      VT - list of vertex sets VT[i] for i > 0 - is a vertex set of i-th connected component
      VT[0] - is a vertex set of the spanning forest (or during the algorithm list of visited vertices).
      """
      VT = [set([])]
      for v in self.graph:
        if v not in VT[0]:      # v is not visited
          VT[0].add(v)
          VT.append(set([v]))   # statring point of new conected component
          DFS(v)
      return VT

    def preorder(self, v, visited=None):
        # first we print a given vertex, then we traverse the subtree rooted in it
        """
        Prints the vertices of the graph in preorder traversal starting from vertex v.
        """
        if visited is None:
            visited = set()

        # Visit the current node
        visited.add(v)
        print(v, end=' ')

        # Recursively visit all unvisited neighbors
        for neighbor in self.graph[v]:
            if neighbor not in visited:
                self.preorder(neighbor, visited)


    def postorder(self, v, visited=None):
        # postorder — first we traverse the subtree rooted a given vertex, then we list the vertex
        """
        Prints the vertices of the graph in postorder traversal starting from vertex v.
        """
        if visited is None:
            visited = set()

        # Mark the current node as visited
        visited.add(v)

        # Recursively visit all unvisited neighbors first
        for neighbor in self.graph[v]:
            if neighbor not in visited:
                self.postorder(neighbor, visited)

        # Print the current node after visiting its neighbors
        print(v, end=' ')


    def connected_components_graphs(self):
        """
        Returns a list of Graph objects, each representing a connected component of the original graph.
        """
        components = self.connected_components()  # Get the list of connected components as vertex sets
        component_graphs = []

        for component in components[1:]:  # Skip the first element as it contains all vertices
            subgraph = Graph()            # Create a new Graph instance
            for vertex in component:
                subgraph.graph[vertex] = self.graph[vertex]
#           equivalent
#                subgraph.add_vertex(vertex)
#                for neighbor in self.graph[vertex]:
#                    if neighbor in component:
#                        subgraph.add_edge([vertex, neighbor])
            component_graphs.append(subgraph)

        return component_graphs


    def ConnectedComponentsBFS(self):
        """
        Connected components via BFS (homework, improved) - returns list of graphs
        """
        visited = set()  # keep track of visited vertices
        components = []  # store connected components

        for start_vertex in self.graph:
            if start_vertex not in visited:
                # if vertex was not visited, create a new component
                component = set()
                queue = deque([start_vertex])  # BFS queue

                while queue:
                    vertex = queue.popleft()    # take the vertex from the bottom
                    if vertex not in visited:
                        visited.add(vertex)
                        component.add(vertex)
                        # add all unvisited neighbors to the queue
                        for neighbor in self.graph[vertex]:
                            if neighbor not in visited:
                                queue.append(neighbor)
                # store the current connected component (as graph)
                subgraph = Graph()            # Create a new Graph instance
                for vertex in component:
                  subgraph.graph[vertex] = self.graph[vertex]
                components.append(subgraph)
        return components

    def ConnectedComponentsGraphs(self):
        """
        Connected components via DFS nonrecursive (homework, corrected) -
        returns list connected subgraphs
        """
        con_com = []
        visited = set()

        def dfs(v, component):
            stack = [v]
            while stack:
                u = stack.pop()
                if u not in visited:
                    visited.add(u)
                    component[u] = [neighbor for neighbor in self.graph[u] ] # correction: if neighbor not in visited]
                    stack.extend(component[u])

        # back to the main attraction
        for vertex in self.graph:
            if vertex not in visited:
                component = {}                  # creating a new component as an adj list
                dfs(vertex, component)          # find all vertices in this component
                con_com.append(Graph.from_dict(component))  #converting to Graph object and append

        return con_com


    @staticmethod
    def random_bipartite_graph(m, n, p):
        """
        Generates a random bipartite graph with m + n vertices.
        The two sets U and V have m and n vertices respectively.
        Each edge between a vertex in U and a vertex in V is included with probability p.
        """
        import random
        bipartite_graph = Graph()
        U = [f"U{i}" for i in range(1, m + 1)]  # Label vertices in set U as U1, U2, ..., Um
        V = [f"V{i}" for i in range(1, n + 1)]  # Label vertices in set V as V1, V2, ..., Vn

        # Add vertices to the graph
        for vertex in U + V:
            bipartite_graph.add_vertex(vertex)

        # Add edges between vertices in U and V with probability p
        for u in U:
            for v in V:
                if random.random() < p:
                    bipartite_graph.add_edge([u, v])

        return bipartite_graph


    def distance(self, v):
      """
      Computes distances from vertex v to each vertex reachable from v.
      It uses a BFS approach.
      Result is given as a dictionary of distances
      """
      dist = {v:0}    # starting point of a dictionary
      queue = [v]
      while len(queue) > 0:
        u = queue.pop(0)
        for w in self.graph[u]:
          if not w in dist:
            dist[w] = dist[u] + 1
            queue.append(w)
      return dist

    def strong_connected_components(self):
        """
        Finds strongly connected components (SCCs) using Kosaraju's algorithm.
        Returns a list of strongly connected components, each component is a set of vertices.
        """
        def dfs(v, visited, stack):
            visited.add(v)
            for neighbor in self.graph[v]:
                if neighbor not in visited:
                    dfs(neighbor, visited, stack)
            stack.append(v)

        def reverse_graph():
            reversed_graph = Graph()
            for v in self.graph:
                for neighbor in self.graph[v]:
                    reversed_graph.add_arc([neighbor, v])  # Reverse the direction of the edges
            return reversed_graph

        def dfs_reverse(v, visited, component):
            visited.add(v)
            component.add(v)
            for neighbor in reversed_graph.graph[v]:
                if neighbor not in visited:
                    dfs_reverse(neighbor, visited, component)

        stack = []
        visited = set()

        for vertex in self.graph:
            if vertex not in visited:
                dfs(vertex, visited, stack)

        reversed_graph = reverse_graph()

        visited.clear()
        scc_list = []

        while stack:
            v = stack.pop()
            if v not in visited:
                component = set()
                dfs_reverse(v, visited, component)
                scc_list.append(component)

        return scc_list
    
    def max_matching(self):
        """
        Finds a maximum matching in a bipartite graph using the Hopcroft-Karp algorithm.
        Returns a dictionary representing the matching, where keys are vertices in one partition
        and values are their matched vertices in the other partition.  Unmatched vertices will not
        appear as keys.
        """
        def bfs():
            queue = deque()
            for u in U:
                if match[u] is None:
                    dist[u] = 0
                    queue.append(u)
                else:
                    dist[u] = float('inf')
            dist[None] = float('inf')
            while queue:
                u = queue.popleft()
                if u is not None:
                    for v in self.graph[u]:
                        if dist[match[v]] == float('inf'):
                            dist[match[v]] = dist[u] + 1
                            queue.append(match[v])
            return dist[None] != float('inf')

        def dfs(u):
            if u is not None:
                for v in self.graph[u]:
                    if dist[match[v]] == dist[u] + 1:
                        if dfs(match[v]):
                            match[v] = u
                            match[u] = v
                            return True
                dist[u] = float('inf')
                return False
            return True
        
        # Identify partitions (assuming bipartite graph)
        U = set()
        V = set()
        for u in self.graph:
            if u not in V:
              U.add(u)
              for v in self.graph[u]:
                  V.add(v)

        match = {}  # Matching dictionary
        for u in U:
            match[u] = None
        for v in V:
            match[v] = None
        
        dist = {}  # Distance dictionary
        
        result = 0
        while bfs():
            for u in U:
                if match[u] is None and dfs(u):
                    result += 1
        
        matching = {}
        for u in match:
            if match[u] is not None:
                matching[u] = match[u]
        return Graph(matching)

## Usage of the algorithm

In [7]:
my_graph = {'A': ['1', '3'], 'B': ['1', '2', '3'], 'C': ['2', '4']}
my_graph = Graph(my_graph)
print(my_graph)
print(my_graph.max_matching())

A: 1 3
B: 1 2 3
C: 2 4

C: 2
A: 1
B: 3
1: A
3: B
2: C



In [8]:
graph = Graph.random_bipartite_graph(7, 5, 0.3)
print(graph)
print(graph.max_matching())

U1: V3
U2: V2 V5
U3: V4 V5
U4: V3
U5: V5
U6: V2 V5
U7: V5
V1:
V2: U2 U6
V3: U1 U4
V4: U3
V5: U2 U3 U5 U6 U7

U5: V 5
U4: V 3
U2: V 2
U3: V 4
V2: U 2
V5: U 5
V4: U 3
V3: U 4

